In [6]:
import time

from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By

In [7]:
driver = Chrome()
driver.get("https://books.toscrape.com/")

In [8]:
last_height = driver.execute_script("return document.body.scrollHeight;")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(3)

    new_height = driver.execute_script("return document.body.scrollHeight;")

    if new_height == last_height:
        break
    last_height = new_height

In [25]:
categoryLinksUl = driver.find_element(By.CLASS_NAME, "nav-list" )
categoryListItems = categoryLinksUl.find_elements(By.TAG_NAME, "li")
categoryLinks = [ listItem.find_element(By.TAG_NAME, "a") for listItem in categoryListItems ]
categoryLinksUrl = [ categoryLink.get_property("href") for categoryLink in categoryLinks ]
categoryNames = [ categoryLink.text for categoryLink in categoryLinks ]

In [39]:
books_by_category = {}

for i in range(len(categoryNames)):
    if i > 4:
        break

    category_url = categoryLinksUrl[i]
    driver.get(category_url)
    books = []

    while True:
        booksDivs = driver.find_elements(By.CLASS_NAME, "product_pod")
        booksH3 = [bookDiv.find_element(By.TAG_NAME, "h3") for bookDiv in booksDivs]
        books += [bookH3.find_element(By.TAG_NAME, "a").text for bookH3 in booksH3]

        try:
            next_button = driver.find_element(By.CLASS_NAME, "next").find_element(By.TAG_NAME, "a")
            next_page_url = next_button.get_property("href")
            driver.get(next_page_url)
        except:
            break

    books_by_category[categoryNames[i]] = books

driver.quit()

MaxRetryError: HTTPConnectionPool(host='localhost', port=54178): Max retries exceeded with url: /session/58b4f826ed6c8a516892bf0b6e388eb9/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12cd71bd0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [40]:
import pandas as pd

data = []
for category, books in books_by_category.items():
    for book in books:
        data.append({"Category": category, "Book Title": book})

df = pd.DataFrame(data)

print(df)

Empty DataFrame
Columns: []
Index: []


In [ ]:
import matplotlib.pyplot as plt

category_counts = df['Category'].value_counts()

plt.bar(category_counts.index, category_counts.values, color='skyblue')
plt.title('Number of Books per Category')
plt.xlabel('Category')
plt.ylabel('Number of Books')
plt.show()